# Subword Tokenizer

| **Tokenizer 방식** | **토큰 단위**                      | **vocab size** | **미등록 단어에 대한 가정**                                                                                  |
|---------------------|------------------------------------|----------------|-------------------------------------------------------------------------------------------------------------|
| **사전 기반**       | 알려진 단어/형태소의 결합           | unlimited       | - 알려진 단어/형태소의 결합이라고 가정<br>- 필요한 형태소 분석 가능<br>- 사전에 등록되지 않은 단어는 UNK 처리 |
| **sub-word**        | 알려진 글자 및 sub-word            | fixed           | - 알려진 sub-words로 분해<br>- 예: appear → app + ear<br>- 자주 등장하는 단어를 제대로 인식 가능<br>- UNK의 개수 최소화 |

### 네이버 영화 리뷰 학습

In [4]:
# 네이버 영화 리뷰 데이터 다운로드(캐시 저장) 함수
import urllib.request                                                               # URL에서 파일을 다운로드하는 모듈
import os                                                                           # 경로/폴더 생성 등 파일 시스템 처리 모듈
# 파일 다운로드 함수 : 지정한 URL(origin)의 파일을 토컬 캐시에 저장하고 경로를 반환
def get_file(filename, origin):
    cache_dir = os.path.expanduser('~/.torch/datasets')                             # 캐시 디렉토리 경로 생성
    os.makedirs(cache_dir, exist_ok=True)                                           # 캐시 폴더 없으면 생성
    filepath = os.path.join(cache_dir, filename)                                    # 저장할 파일 전체 파일 경로
    
    if not os.path.exists(filepath):                                                 # 파일이 없으면
        print(f'다운로드 진행중! {origin}')
        urllib.request.urlretrieve(origin, filepath)                                # origin에서 파일을 내려받아 filepath에 저장
        
    return filepath                                                                 # 로컬에 저장한 파일 경로 반환

In [5]:
# NSMC(네이버 영화 리뷰) 학습/테스트 파일 다운로드 경로 확인
ratings_train_path = get_file(
    'ratings_train.txt',
    'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt'
)
ratings_test_path = get_file(
    'ratings_test.txt',
    'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt'
)

ratings_train_path, ratings_test_path

다운로드 진행중! https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
다운로드 진행중! https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt


('C:\\Users\\Playdata/.torch/datasets\\ratings_train.txt',
 'C:\\Users\\Playdata/.torch/datasets\\ratings_test.txt')

In [7]:
# NSMC 데이터 텍스트 파일을 DataFrame 으로 로드 후 확인
import pandas as pd

rating_train_df = pd.read_csv(ratings_train_path, sep='\t')             # 학습 데이터 (탭 구분)
rating_test_df = pd.read_csv(ratings_test_path, sep='\t')               # 테스트 데이터 (탭 구분)

display(rating_train_df)
display(rating_test_df)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


.txt 파일이 id \t document \t label 구조로 되어있다.
결국 테이블 형태로 이루어져있어 read_csv로 읽어올 수 있다.